# Steinkauz

## Notebook: avo_for_density

See the [Steinkauz repository](https://github.com/bjornrommel/steinkauz/avo_for_density) for the code source.

In [ ]:
# Steinkauz

# load module
%reload_ext steinkauz

# set up LaTeX backend
%steinkauz --option setup --prefile mypackages.tex --predir .

# toggle input
###%inputtoggle

In [ ]:
# Python, IPython

# import Python modules
import sys
import numpy as np

# import IPython modules
from IPython import display as dsp
import ipywidgets as ipw
from ipywidgets import interact, interactive, interactive_output

In [ ]:
# Matplotlib

# set default parameters 
import matplotlib as mpl
mpl.rcParams['font.size'] = 11             # same font size as the LaTeX document
mpl.rcParams['text.usetex'] = True         # use latex throughout
mpl.rcParams['ps.usedistiller'] = 'xpdf'   # for production quality

# set API
from matplotlib import pyplot as plt       # use Matlab-like API

# set backend (ipympl integrates interactive features into Jupyter)
%matplotlib ipympl 

In [ ]:
# User-Defined Python 

# import notebook-specific functions
import avo_for_density as afd
import config as cfg

# <center> Elements of Seismic Data Processing </center>

# <center> AVO Inversion for Density </center>

# <center> Björn E. Rommel </center>

In [ ]:
###%steinkauz --option notebook --texdir title --texfile title --noshow VERSION

## The Forward Problem

Commonly, the "forward problem" means simulating a physical process: we start with a model of reality, and then we calculate data based on some mathematical description of a physical law. Here, a plane wave impinging on an interface is partially reflected, partially transmitted, and the respective reflection / transmission coefficients are described by the Zoeppritz equation. Such a coefficient (or commonly called amplitude), in turn, depends on the incidence angle of the plane wave; hence, the term amplitude-versus-angle (AVA) arises.

The Zoeppritz equation depend on the seismic properties P-velocity, S-velocity and density on either side of the interface. For the sake of a better intuitive understand, and simplicity of computations, the Zoeppritz equation are commonly linearized in terms of small differences in seismic properties from their respective "background" (or average) medium. Then, a first-order linearized AVA depends on the normalized contrasts of seismic properties, the P-to-S velocity ratio of the background, and the incidence angles.

### Input of Seismic Properties

Nonetheless, traditionally, seismic properties, not their normalized contrasts, are input, as follows.

Tip: You can skip inputting these seismic properties, if you don't want to see a theoretical AVA curve, but to jump ahead to an AVA inversion.

In [ ]:
# write top halfspace
display(dsp.Markdown("#### Top Halfspace:"))

# define input boxes for vp, vs, and rho in the top halfspace
vp1_box = ipw.BoundedFloatText(description="P-velocity", value=2000, min=0, step=10, max=sys.maxsize)
vs1_box = ipw.BoundedFloatText(description="S-velocity", value=1200, min=0, step=10, max=sys.maxsize)
rho1_box = ipw.BoundedFloatText(description="density", value=2000, min=0, step=10, max=sys.maxsize)
    
# group input boxes into a horizontal row
top_box = ipw.GridBox(
    [vp1_box, vs1_box, rho1_box], 
    layout=ipw.Layout(grid_template_columns="repeat(3, 300px)"))
display(top_box)

# write bottom halfspace
display(dsp.Markdown("#### Bottom Halfspace:"))

# define input boxes for vs, vs, and rho in the bottom halfspace
vp2_box = ipw.BoundedFloatText(description="P-velocity", value=2500, min=0, step=10, max=sys.maxsize)
vs2_box = ipw.BoundedFloatText(description="S-velocity", value=1500, min=0, step=10, max=sys.maxsize)
rho2_box = ipw.BoundedFloatText(description="density", value=2200, min=0, step=10, max=sys.maxsize)

# group input boxes into a horizontal row
bot_box = ipw.GridBox(
    [vp2_box, vs2_box, rho2_box], 
    layout=ipw.Layout(grid_template_columns="repeat(3, 300px)"))
display(bot_box)


# calculate model and S-to-P
def update_model(**kwargs):
    # check all properties are set
    if not np.any([np.isnan(kwargs[prop]) for prop in ['vp1', 'vs1', 'rho1', 'vp2', 'vs2', 'rho2']]):
        # set up the model
        cfg.mod = [
            (kwargs['vp2'] - kwargs['vp1']) / ((kwargs['vp1'] + kwargs['vp2']) / 2.),
            (kwargs['vs2'] - kwargs['vs1']) / ((kwargs['vs1'] + kwargs['vs2']) / 2.),
            (kwargs['rho2'] - kwargs['rho1']) / ((kwargs['rho1'] + kwargs['rho2']) / 2.)]
        # calculate the S-to-P velocity ratio
        cfg.vsp = (kwargs['vs1'] + kwargs['vs2']) / (kwargs['vp1'] + kwargs['vp2'])

# interact with input boxes and call for AVA calculation and curve preparation
interactive_output(
    update_model,
    {'vp1': vp1_box, 'vs1': vs1_box, 'rho1': rho1_box, 'vp2': vp2_box, 'vs2': vs2_box, 'rho2': rho2_box})

### S-to-P Velocity Ratio

The S-to-P velocity ratio, however, is required for both the forward and inverse problem. If all seismic properties are input above, the S-to-P velocity ratio is automatically populated, but it can be overridden in the following; otherwise, the S-to-P velocity ratio must be input next.  

In [ ]:
# define an input box for the S-to-P velocity ratio
vsp_box = ipw.BoundedFloatText(description="S-to-P velocity ratio", value=cfg.vsp, min=0, step=0.1, max=sys.maxsize)

# interact with vsp box
@interact(vsp=vsp_box)
def vsp_update(**kwargs):
    # update S-to-P velocity ratio
    cfg.vsp = kwargs['vsp']


# update the default value of the input box
def value_update(*args):
    vsp_box.value = cfg.vsp
vp1_box.observe(value_update, 'value')
vs1_box.observe(value_update, 'value')
vp2_box.observe(value_update, 'value')
vs2_box.observe(value_update, 'value')

### AVA Curve and Amplitude Input

Finally, the theoretical AVA curve can be calculated and displayed. 

In [ ]:
# calculate and display the AVA curve
def update_curve(**kwargs):
    # check all properties are set
    if not np.any([np.isnan(kwargs[prop]) for prop in ['vp1', 'vs1', 'rho1', 'vp2', 'vs2', 'rho2']]):
        # calculate AVA and prepare the curve
        afd.comp_ava(mod=cfg.mod, vsp=cfg.vsp)
# interact with input boxes and call for AVA calculation and curve preparation
ava_box = interactive_output(
    update_curve,
    {'vp1': vp1_box, 'vs1': vs1_box, 'rho1': rho1_box, 'vp2': vp2_box, 'vs2': vs2_box, 'rho2': rho2_box})

# display AVA curve
display(ava_box)

## The Inverse Problem

The "inverse problem" is the opposite, that is inferring a model from available data. "Bayes's theory" of inversion aims to optimize that inference: based on already a-priori available information about the model, an estimate of the uncertainty of such prior information, and further based on an estimate of data quality, Bayes theory provides the optimal a-posteriori information possible.

In AVA inversion, here, the normalized contrasts make up the "model" in a Bayesian sense: as said, the properties to invert for. Hence, in a Bayesian context, the "prior model" comprises all information we have already derived from, e.g., a velocity analysis; and the "prior uncertainty" is the uncertainty with which we can perform a velocity analysis. 

The "parameters" in an AVA inversion are the P-to-S velocity ratio and the incidence angles. For now, let's assume, we know these parameters with zero uncertainty, although, admittedly, such accuracy will never be possible in practice.

Then, the "data" are the reflection / transmission coefficients. Here, the terms coefficients and "amplitude" are often mixed up: AVA-inversion requires the coefficients or normalized amplitudes, that is the amplitude ratio of a wave after and before reflection / transmission. True-amplitude processing seems elusive in practice, but that uncertainty is part of the "data uncertainty."

### Data

The data -- amplitude over incidence angle -- can be input directly in the above figure.

Tip: The order in which the data are input does not matter. You only need to click on the respective point in the display; the actual cursor position is shown at the bottom of the figure.

### Prior Model Uncertainty

Let's take the prior model as above, and input only the prior uncertainty. Admittedly, estimating prior uncertainty is in itself another Bayesian problem. 

Tip: However, recall in an heuristic approach, a distribution of random numbers is almost entirely contained within 3 times its standard deviation (STD): so, estimate the maximum error of, e.g., our velocity analysis and devide that value by 3 to obtain the prior uncertainty expressed in terms of its standard deviation. Not perfect, but good enough for a first gestimate.

In [ ]:
# write subtitle for the top halfspace
display(dsp.Markdown("#### Top Halfspace:"))

# write input boxes for seismic properties in the top halfspace
dvp1_box = ipw.BoundedFloatText(description="P-velocity +/-", value=25, min=0, step=1, max=sys.maxsize)
dvs1_box = ipw.BoundedFloatText(description="S-velocity +/-", value=15, min=0, step=1, max=sys.maxsize)
drho1_box = ipw.BoundedFloatText(description="density +/-", value=20, min=0, step=1, max=sys.maxsize)

# line up input boxes for the top halfspace and display
dtop_box = ipw.GridBox(
    [dvp1_box, dvs1_box, drho1_box], 
    layout=ipw.Layout(grid_template_columns="repeat(3, 300px)"))
display(dtop_box)

# write subtitle for the bottom halfspace
display(dsp.Markdown("#### Bottom Halfspace:"))

# write input boxes for seismic properties in the bottom halfspace

dvp2_box = ipw.BoundedFloatText(description="P-velocity +/-", value=25, min=0, step=1, max=sys.maxsize)
dvs2_box = ipw.BoundedFloatText(description="S-velocity +/-", value=15, min=0, step=1, max=sys.maxsize)
drho2_box = ipw.BoundedFloatText(description="density +/-", value=20, min=0, step=1, max=sys.maxsize)

# line up input boxes for the bottom halfspace and display
dbot_box = ipw.GridBox(
    [dvp2_box, dvs2_box, drho2_box], 
    layout=ipw.Layout(grid_template_columns="repeat(3, 300px)"))
display(dbot_box)

# update prior model
def update_prior(**kwargs):
    # assign input
    cfg.est = [
        np.sqrt(kwargs['dvp1'] ** 2 + kwargs['dvp2'] ** 2) / 2.,
        np.sqrt(kwargs['dvs1'] ** 2 + kwargs['dvs2'] ** 2) / 2.,
        np.sqrt(kwargs['drho1'] ** 2 + kwargs['drho2'] ** 2) / 2.]
    
# interact with all input boxes
interactive_output(
    update_prior, 
    {'dvp1': dvp1_box, 'dvs1': dvs1_box, 'drho1': drho1_box, 'dvp2': dvp2_box, 'dvs2': dvs2_box, 'drho2': drho2_box})

### Data Uncertainty

"Data uncertainty", in a Bayesian sense, is the standard deviation (STD) of ambient noise contaminating the data, thereby assuming noise samples are normally distributed.

However, the signal-to-noise ratio (SNR) is more popular. Here, the signal-to-noise ratio (SNR) is defined as the ratio of a wavelet's amplitude (A) and the standard deviation (STD) of ambient noise. The graph shows the superposition of a triangular wavelet with amplitude 1.0 and a realization of random noise with the requested STD (by definition, STD = A * SNR). 

Tip: Adjust the signal-to-noise ratio until the amount of noise appear similar to your data.

In [ ]:
# Signal-To-Noise Ratio

# define the input box for the signal-to-noise ratio
snr_box = ipw.BoundedFloatText(description="signal-to-noise", value=0.05, step=0.01)

# display wavelet with ambient noise at the given signal-to-noise ratio
def snr_update(**kwargs):
    # update signal-to-noise ratio
    cfg.snr = kwargs['snr']
    # display 
    afd.plot_wavelet()
    
# interact with the signal-to-noise ratio box
out = ipw.interactive_output(snr_update, {'snr': snr_box})
display(snr_box)
display(out)


# define an update button
def button_action(*args):
    # wake up Jupyter
    pass

# interact with update button
button =  ipw.Button(description = 'update', button_style = 'info', tooltip = 'update data uncertainty')
display(button)
button.on_click(button_action)

In [ ]:
# prepare final pdf file
###%steinkauz --option printout --texdir . --texfile avo_for_density

# WORK IN PROGRESS!!!

In [ ]:
# clean
%steinkauz -o clean

In [ ]:
# input toggle
###%inputtoggle